# MiniCaml: Funzioni a dominio Finito.

Una funzione (unaria e non ricorsiva) a *dominio finito* è una funzione che è definita solo per un numero
finito di elementi. Ad esempio si consideri la seguente funzione con una sintassi nello stile di OCaml

    fun y -> y + 1 for y in [0; 1; 2; 3; 4];;

La funzione è definita solamente per valori del parametro attuale che appartengono all’insieme `{0, 1, 2, 3, 4}`,
insieme che è calcolato al momento della definizione della funzione stessa.

## Es. 1: Sintassi

Si estenda l'AST di Minicaml (riportato sotto) con un nuovo costrutto per le funzioni a dominio finito. Quante e quali sottoespressioni deve contenere?

### Soluzione:

Una definizione di funzione (anonima) con dominio finito, come 

    fun y -> y + 1 for y in [0; 1; 2; 3; 4]
è simile a una normale definizione di funzione aninima in MiniCaml, come 

    fun y -> y + 1
che si formalizza in sintassi astratta con il costruttore `Fun` (`Fun ("y", Sum(Ide(y), EInt(1))`).
Il nuovo costrutto che vogliamo introdurre sarà quindi simile a `Fun`, possiamo chiamarlo `FunFD`, e come `Fun` richiederà una coppia `ide * exp`. Oltre all'argomento e al corpo della funzione, l'espressione che ci interessa contiene però anche un dominio, che possiamo formalizzare come una lista di espressioni `exp list`. 

Se intendiamo il tipo `exp` come un albero, allora un'espressione costruita con `FunFD` ha un numero non deciso a priori di figli, proprio perchè dentro la definizione del dominio ci può essere un numero arbitrario (ma finito) di sottoespressioni.

### Albero della sintassi astratta

In [2]:
(* Identificatori *)
type ide = string;;

(* Abstract Expressions = espressioni nella sintassi astratta, 
   compongono l'Albero di Sintassi Astratta *)
type exp = 
    | EInt of int
    | CstTrue 
    | CstFalse
    | EString of string
    | Den of ide
    (* Operatori binari da interi a interi*)
    | Sum of exp * exp
    | Diff of exp * exp
    | Prod of exp * exp
    | Div of exp * exp
    (* Operatori da interi a booleani *)
    | IsZero of exp
    | Eq of exp * exp
    | LessThan of exp*exp
    | GreaterThan of exp*exp
    (* Operatori su booleani *)
    | And of exp*exp
    | Or of exp*exp
    | Not of exp
    (* Controllo del flusso, funzioni *)
    | IfThenElse of exp * exp * exp
    | Let of ide * exp * exp
    | Letrec of ide * ide  * exp * exp
    | Fun of ide * exp
    | FunFD of ide * exp * (exp list)
    | Apply of exp * exp

type ide = string


type exp =
    EInt of int
  | CstTrue
  | CstFalse
  | EString of string
  | Den of ide
  | Sum of exp * exp
  | Diff of exp * exp
  | Prod of exp * exp
  | Div of exp * exp
  | IsZero of exp
  | Eq of exp * exp
  | LessThan of exp * exp
  | GreaterThan of exp * exp
  | And of exp * exp
  | Or of exp * exp
  | Not of exp
  | IfThenElse of exp * exp * exp
  | Let of ide * exp * exp
  | Letrec of ide * ide * exp * exp
  | Fun of ide * exp
  | FunFD of ide * exp * exp list
  | Apply of exp * exp


## Es 2: Valori e Tipi

Estendere la nozione di valore (`evT`), di tipo (`tname`) per includere le funzioni a dominio finito, e aggiornare `getype` e `typecheck` di conseguenza.

### Soluzione:

Una normale funzione anonima (costruita con `Fun`) viene valutata come un valore `Closure`, che contiene una tripla argomento, corpo e ambiente di definitione. Allo stesso modo una funzione a dmonio finito possiamo valutarla come un nuovo valore `FDClosure`, che conterrà le stesse informazioni di una closure, e in aggiunta anche un dominio.

Ci sono due alternative su come includere il dominio dentro una `FDClosure`:
- Includere una lista di **valori**, `evT list`, che quindi vengono valutati una sola volta, quando la funzione viene definita.
- Includere una lista di espressioni, la stessa presente nella definzione `FunFD`. Come precisato dalla consegna, il valore di queste espressioni che compongono il dominio deve essere valutato nell'ambiente di definizione, non in quello di esecuzione. Quindi ogni volta che verrà valutata una applicazione di una funzione a dominio finito bisognerà valutare le espressioni del dominio nell'ambiente presente dentro closure.

Entrambe le soluzioni sono corrette, ma la seconda ci obbliga a ricalcolare più volte gli stessi valori nello stesso ambientem, quindi sceglieremo la prima. 

Per il tipo, possiamo aggiunge un nuovo tipo a `tname`, e aggiornare le funzioni `getType` e `type_check` trivialmente. Alternativamente, possiamo anche decidere che una `FDClosure` debba essere di tipo `TClosure`, il resto del codice non cambia. Notate che il concetto di "tipo" di una closure non viene mai usato nell'interprete, è presente solo per completezza, quindi non c'è nessun motivo per preferire una scelta all'altra. 

Al contrario, se anzichè voler introdurre un nuovo costrutto funzionale stessimo introducendo un nuovo tipo di dato, come i float, i record, gli insiemi o quant'altro, definire dei nuovi tipi appropriati sarebbe fondamentale, perchè l'interprete quando manipola i dati chiama sempre delle primitive, e le primitive chiamano il typechecker.

### Ambiente

In [3]:
(* Ambiente polimorfo *)
type 't env = ide -> 't

(* Evaluation types = tipi  esprimibili *)
type evT = 
    | Int of int 
    | Bool of bool 
    | String of string 
    | Closure of ide * exp * evT env 
    | FDClosure of ide * exp * evT env * evT list
    | RecClosure of ide * ide * exp * evT env
    | UnBound
    

(* Ambiente vuoto *)
let emptyenv = function x -> UnBound

(* Binding fra una stringa x e un valore primitivo evT *)
let bind (s: evT env) (x: ide) (v: evT) = 
    function (i: ide) -> if (i = x) then v else (s i)

(* I tipi *)
type tname =  TInt | TBool | TString | TClosure | TRecClosure | TFDClosure | TUnBound

(* Funzione da evT a tname che associa a ogni valore il suo descrittore di tipo  *)
let getType (x: evT) : tname =
    match x with
    | Int(n) -> TInt
    | Bool(b) -> TBool
    | String(s) -> TString
    | Closure(i,e,en) -> TClosure
    | FDClosure _ -> TFDClosure
    | RecClosure(i,j,e,en) -> TRecClosure
    | UnBound -> TUnBound

type 't env = ide -> 't


type evT =
    Int of int
  | Bool of bool
  | String of string
  | Closure of ide * exp * evT env
  | FDClosure of ide * exp * evT env * evT list
  | RecClosure of ide * ide * exp * evT env
  | UnBound


val emptyenv : 'a -> evT = <fun>


val bind : evT env -> ide -> evT -> ide -> evT = <fun>


type tname =
    TInt
  | TBool
  | TString
  | TClosure
  | TRecClosure
  | TFDClosure
  | TUnBound


val getType : evT -> tname = <fun>


### Typechecking

In [12]:
(* Type-checking *)
let typecheck ((x, y) : (tname*evT)) = 
    match x with
    | TInt -> (match y with 
               | Int(u) -> true
               | _ -> false
               )
    | TBool -> (match y with 
                | Bool(u) -> true
                | _ -> false
                )
    | TString -> (match y with
                 | String(u) -> true
                 | _ -> false
                 )
    | TClosure -> (match y with
                   | Closure(i,e,n) -> true
                   | _ -> false
                   )
    | TFDClosure -> (match y with
                   | FDClosure _ -> true
                   | _ -> false
                   )
    | TRecClosure -> (match y with
                     | RecClosure(i,j,e,n) -> true
                     | _ -> false
                     )
    |TUnBound -> (match y with
                 | UnBound -> true
                 | _ -> false
                 )

(* Errori a runtime *)
exception RuntimeError of string

val typecheck : tname * evT -> bool = <fun>


exception RuntimeError of string


## Es 3: Interprete

Estendere la funzione di valutazione (`eval`) per includere le funzioni a dominio finito.

**HINT**: Bisogna estende due passaggi dell'interprete, specificando come funziona la *definizione* di una funzione a dominio finito, e come funziona *l'applicazione*. Ricordate che i valori del dominio vanno valutati a tempo di definizione.

### Soluzione

Dobbiamo estendere l'interprete in due punti diversi del codice, la definizione di una funzione FD e l'applicazione di una funzione FD.

Per la definizione, abbiamo già deciso che un'espressione `FunFD` dovrebbe essere valutata come una `FDClosure`. Dentro la `FDClosure` mettiamo l'argomento, il corpo e l'ambiente presente al momento della definizione, cioè `s`. Quando dopo continueremo la valutazione ed eseguiremo altre chiamate ricorsive di `eval`, il parametro `s` sarà cambiato, poichè indica l'ambiente che viene di volta in volta aggiornato. Per questo motivo dentro la closure dobbiamo salvare l'ambiente al momento della definizione, per "preservarlo" dai possibili cambiamenti che avverrano continuando l'esecuzione.

Dentro la `FDClosure` va incluso anche il dominio, ovvero una lista di valori. Per trasformare la lista di espressioni di `FunFD` nella lista di valori di `FDClosure` possiamo seguire due strade:
- Usare la funzione List.map, mappando quindi una lista di espressioni in una lista di valori, uno per uno
- Definire una eval ad hoc che valuti tutte le espressioni di una lista, tipo
      let rec evalAll s list = 
          match list with
          | [] -> []
          | e::es -> (eval e s) :: (evalAll s es)

Entrambe le scelte sono giuste, in generale usando funzioni di ordine superiore come `List.map` si scrive meno codice, quindi è meno probabile fare errori.

Per l'applicazione, presa un'espressione `Apply(eF, eArg)`, bisogna prima di tutto valutare `eF` per ottenere un valore (parte del codice già presente), e poi fare matching su questo valore. Noi dobbiamo implementare il caso in cui `eval eF` sia una `FDClosure(arg, fbody, fDecEnv, domain)`. Come per gli altri casi, valutiamo l'**espressione** `eArg` per ottenere il **valore** `aVal`, che è il nostro parametro attuale. Dobbiamo quindi controllare che `aVal` sia nel dominio della `FDClosure`, che si può fare con un'opportuna funzione di libreria come `List.find_opt` o `List.exists`. Se `aVal` non appartiene al dominio, lanciamo un errore. Se `aVal` appartiene al dominio, dobbiamo procedere con una normale esecuzione della funzione: partiamo dall'ambiente di definizione della closure, leghiamo il nome `arg` al valore `aVal`, ed eseguiamo in questo ambiente il corpo della funzione.

### Takeaway:
- Per svolgere gli esercizi su Minicaml, è fondamentale studiare i costrutti già presenti e prenderli come modello.
- `eval` gestisce il "controllo" (`if` e funzioni), mentre le primitive gestiscono i "dati", effettuando anche il type checking dinamico.
- una closure deve contenere tutte le informazioni necessarie a eseguire una funzione
- Eseguire una funzione vuol dire semplicemente eseguire il body in un nuovo abiente, che è l'ambiente di definizione più l'associazione `parametro formale = parametro attuale`.

### Primitive

In [13]:
(* PRIMITIVE del linguaggio *)

(* Controlla se un numero è zero *)
let is_zero(x) = match (typecheck(TInt,x),x) with
    | (true, Int(v)) -> Bool(v = 0)
    | (_, _) -> raise ( RuntimeError "Wrong type")

(* Uguaglianza fra interi *)
let int_eq(x,y) =   
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v = w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Somma fra interi *)	   
let int_plus(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v + w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Differenza fra interi *)
let int_sub(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v - w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Prodotto fra interi *)
let int_times(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v * w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")
    
(* Divisione fra interi *)
let int_div(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> 
                    if w<>0 then Int(v / w)
                            else raise (RuntimeError "Division by zero")
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni di confronto *)
let less_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v < w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let greater_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v > w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni logiche *)
let bool_and(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v && w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_or(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v || w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_not(x) = 
    match (typecheck(TBool,x), x) with
    | (true, Bool(v)) -> Bool(not(v))
    | (_,_) -> raise ( RuntimeError "Wrong type")

val is_zero : evT -> evT = <fun>


val int_eq : evT * evT -> evT = <fun>


val int_plus : evT * evT -> evT = <fun>


val int_sub : evT * evT -> evT = <fun>


val int_times : evT * evT -> evT = <fun>


...

val less_than : evT * evT -> evT = <fun>


val greater_than : evT * evT -> evT = <fun>


val bool_and : evT * evT -> evT = <fun>


val bool_or : evT * evT -> evT = <fun>


val bool_not : evT -> evT = <fun>


### Interprete

In [24]:
(* Interprete *)
let rec eval (e:exp) (s:evT env) : evT = 
    match e with
    | EInt(n) -> Int(n)
    | CstTrue -> Bool(true)
    | CstFalse -> Bool(false)
    | EString(s) -> String(s)
    | Den(i) -> (s i)

    | Prod(e1,e2) -> int_times((eval e1 s), (eval e2 s))
    | Sum(e1, e2) -> int_plus((eval e1 s), (eval e2 s))
    | Diff(e1, e2) -> int_sub((eval e1 s), (eval e2 s))
    | Div(e1, e2) -> int_div((eval e1 s), (eval e2 s))

    | IsZero(e1) -> is_zero (eval e1 s)
    | Eq(e1, e2) -> int_eq((eval e1 s), (eval e2 s))
    | LessThan(e1, e2) -> less_than((eval e1 s),(eval e2 s))
    | GreaterThan(e1, e2) -> greater_than((eval e1 s),(eval e2 s))

    | And(e1, e2) -> bool_and((eval e1 s),(eval e2 s))
    | Or(e1, e2) -> bool_or((eval e1 s),(eval e2 s))
    | Not(e1) -> bool_not(eval e1 s)

    | IfThenElse(e1,e2,e3) -> 
        let g = eval e1 s in 
            (match (typecheck(TBool,g),g) with
            |(true, Bool(true)) -> eval e2 s
            |(true, Bool(false)) -> eval e3 s
            |(_,_) -> raise ( RuntimeError "Wrong type")
            )
    | Let(i, e, ebody) -> eval ebody (bind s i (eval e s))
    | Fun(arg, ebody) -> Closure(arg,ebody,s)
    | FunFD(arg, ebody, edomain) -> 
        let domain = List.map (fun e -> eval e s) edomain
        in if List.for_all (fun v -> typecheck(TInt,v)) domain
            then FDClosure(arg, ebody, s, domain)
            else raise ( RuntimeError "wrong type")
    | Letrec(f, arg, fBody, leBody) ->
        let benv = bind (s) (f) (RecClosure(f, arg, fBody,s)) in
            eval leBody benv
    | Apply(eF, eArg) ->
        let fclosure = eval eF s in 
            (match fclosure with 
            | Closure(arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in 
                let aenv = bind fDecEnv arg aVal in 
                eval fbody aenv 
            | RecClosure(f, arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in
                let rEnv = bind fDecEnv f fclosure in
                let aenv = bind rEnv arg aVal in 
                eval fbody aenv
            | FDClosure(arg, fbody, fDecEnv, domain) -> 
                let aVal = eval eArg s in 
                if List.exists (fun v -> v = aVal) domain
                then
                    let aenv = bind fDecEnv arg aVal in 
                    eval fbody aenv 
                else 
                    raise (RuntimeError "input outside domain")
            | _ -> raise ( RuntimeError "Wrong type")
            )

val eval : exp -> evT env -> evT = <fun>


In [36]:
let eFun = FunFD("x", Sum(Den("x"), EInt(1)), [EInt(0); EInt(1); Sum(EInt(1), EInt(1))]);;

let eAppl = Apply(e1, EInt(1));;

eval eAppl emptyenv;;

val eFun : exp =
  FunFD ("x", Sum (Den "x", EInt 1), [EInt 0; EInt 1; Sum (EInt 1, EInt 1)])


val eAppl : exp =
  Apply
   (FunFD ("x", Sum (Den "x", EInt 1),
     [EInt 0; EInt 1; Sum (EInt 1, EInt 1)]),
   EInt 1)


- : evT = Int 2
